# [ E-6 ] 프로젝트: 멋진 작사가 만들기

## 1. 데이터 다운로드

wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기

## 2. 데이터 읽어오기

**glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장한다.**

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()                      # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['"Don\'t worry about a thing,', "'Cause every little thing gonna be all right.", 'Singin\': "Don\'t worry about a thing,']


## 3. 데이터 정제

> - preprocess_sentence() 함수를 활용해 데이터를 정제한다.
> - 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외한다. 

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 텐서플로우
import os

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if len(sentence) > 15: continue    # 길이가 15 이상인 문장은 건너뜁니다.     
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다. 
        
    
    if idx > 9: break   # 일단 문장 10개만 확인해 본다.
        
    print(sentence)

**전처리를 위해 정규표현식 이용한 필터링을 사용한다.**

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


**정제함수를 활용해서 정제 데이터를 구축한다.**

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]               # 앞의 10문장 출력

['<start> don t worry about a thing , <end>',
 '<start> cause every little thing gonna be all right . <end>',
 '<start> singin don t worry about a thing , <end>',
 '<start> cause every little thing gonna be all right ! rise up this mornin , <end>',
 '<start> smiled with the risin sun , <end>',
 '<start> three little birds <end>',
 '<start> perch by my doorstep <end>',
 '<start> singin sweet songs <end>',
 '<start> of melodies pure and true , <end>',
 '<start> sayin , this is my message to you ou ou singin don t worry bout a thing , <end>']

**정제된 데이터를 토큰화 한다(corpus를 텐서로 변환).**

토큰화(Tokenize): 문장을 일정한 기준으로 자르는 과정

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen =15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   37   15 ...    0    0    0]
 [   2   67  133 ...    0    0    0]
 [   2 1551   37 ...    0    0    0]
 ...
 [   5   22  712 ... 4178    4    3]
 [   5   22  712 ... 4178    4    3]
 [   5   22  712 ... 4178   20    3]] <keras_preprocessing.text.Tokenizer object at 0x7fc138b8a3c8>


> **check point)**  "maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다."  어떻게 조정하는가?
> - "tf.keras.preprocessing.sequence.pad_sequences"에 maxlen 설정하기
>

**소스문장과 타겟문장을 생성한다.**

"start" 가 문장의 시작에 더해진 입력 데이터(문제지)와, "end" 가 문장의 끝에 더해진 출력 데이터(답안지)가 필요하며, 이는 문장 데이터만 있으면 만들어낼 수 있다.

In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])             # 소스 문장 확인
print(tgt_input[0])             # 타겟 문장 확인

[  2  37  15 717 113   9 181   4   3   0   0   0   0   0]
[ 37  15 717 113   9 181   4   3   0   0   0   0   0   0]


0은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우, 0으로 패딩을 채워넣은 것이다.

**데이터셋 객체를 생성한다.**

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 개수와, 여기 포함되지 않은 0:<pad>를 포함

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)  # corpus 텐서를 dataset 객체로 변환
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

Batch size는 256, LSTM에 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스 출력값 14의 Dataset이 생성되었다.

## 4. 평가 데이터셋 분리

> - tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리한다.
> - 단어장의 크기는 12,000 이상으로 설정한다.
> - 총 데이터의 20%를 평가 데이터셋으로 사용한다.
> - 만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토해 본다.

**훈련데이터와 평가데이터를 분리한다.**

In [9]:
from sklearn.model_selection import train_test_split

# train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


**옵션 값**

- test_size:  테스트 셋 구성의 비율을 나타낸다. train_size의 옵션과 반대 관계에 있는 옵션 값이며, 주로 test_size를 지정해 준다. 0.2는 전체 데이터 셋의 20%를 test (validation) 셋으로 지정하겠다는 의미이며, 참고로 default 값은 0.25이다.
- shuffle:  split을 해주기 이전에 섞을건지 여부이다. default=True이며, 보통은 default 값으로 둔다.
- stratify:  classification을 다룰 때 매우 중요한 옵션값이다. stratify 값을 target으로 지정해주면 각각의 class 비율(ratio)을 train / validation에 유지해 준다.(즉, 한 쪽에 쏠려서 분배되는 것을 방지한다) 만약 이 옵션을 지정해 주지 않고 classification 문제를 다룬다면, 성능의 차이가 많이 날 수 있습니다. default=None 이다.
- random_state:  세트를 섞을 때 해당 int 값을 보고 섞으며, 하이퍼 파라미터를 튜닝시 이 값을 고정해두고 튜닝해야 매번 데이터셋이 변경되는 것을 방지할 수 있다.

**추가) numpy random에서도 데이터 분리 가능**

1. scikit-learn 라이브러리 model_selection 클래스의 train_test_split 함수를 이용하여 train, test set 분할하기
2. numpy random 클래스의 permutation() 함수를 이용하여 train, test set 분할하기
3. numpy random 클래스의 choice() 함수를 이용하여 train, test set 분할하기
4. numpy random 클래스의 shuffle() 함수를 이용하여 train, test set 분할하기


출처: https://rfriend.tistory.com/519 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

## 5. 인공지능 만들기

> - 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계한다.

**모델 설계하기**

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

print("완료")

완료


> **check poin)** 적당한 embedding size와 hidden_size는 어떻게 정하는가?
> - embedding size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기이다.
> - hidden size는 LSTM 레이어의 hidden state의 차원수 이다.

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: id=385, shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.83470111e-04, -2.33849103e-04,  1.33321402e-04, ...,
         -4.80654235e-05, -1.51777795e-05,  1.70678337e-04],
        [ 3.48550559e-04, -2.96735932e-04,  4.01951082e-04, ...,
         -1.45048252e-04, -2.45581614e-04,  1.97778063e-04],
        [ 3.70774651e-04, -3.91234120e-04,  6.19361235e-04, ...,
         -9.63204584e-05, -3.51205323e-04, -7.79941547e-05],
        ...,
        [ 6.49349357e-04, -4.47583298e-04,  3.27827467e-04, ...,
         -3.61214625e-04, -6.17709593e-04, -9.49099427e-04],
        [ 6.69393106e-04, -4.14761540e-04,  2.58772314e-04, ...,
         -2.94661673e-04, -3.94051633e-04, -9.70066874e-04],
        [ 7.26587954e-04, -3.85876017e-04,  1.86687437e-04, ...,
         -1.42125864e-04, -1.16386487e-04, -9.72599955e-04]],

       [[ 1.83470111e-04, -2.33849103e-04,  1.33321402e-04, ...,
         -4.80654235e-05, -1.51777795e-05,  1.70678337e-04],
        [ 2.75023311e-04, -1.97330388

**결과)** shape=(256, 14, 12001)에서 256은 배치 사이즈, 14는 입력-출력 시퀀스 길이, 12001은 Dense 레이어의 출력 차원수이다. 

In [14]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5028000   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8004000   
_________________________________________________________________
dense_2 (Dense)              multiple                  12013001  
Total params: 28,117,257
Trainable params: 28,117,257
Non-trainable params: 0
_________________________________________________________________


**인공지능 학습시키기**

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)                   # 10으로 설정

Epoch 1/10
686/686 [==============================] - 89s 130ms/step - loss: 3.6345
Epoch 2/10
686/686 [==============================] - 90s 132ms/step - loss: 3.1671
Epoch 3/10
686/686 [==============================] - 99s 144ms/step - loss: 2.9763
Epoch 4/10
686/686 [==============================] - 98s 142ms/step - loss: 2.8269
Epoch 5/10
686/686 [==============================] - 98s 143ms/step - loss: 2.6977
Epoch 6/10
686/686 [==============================] - 98s 143ms/step - loss: 2.5822
Epoch 7/10
686/686 [==============================] - 98s 143ms/step - loss: 2.4745
Epoch 8/10
686/686 [==============================] - 98s 143ms/step - loss: 2.3729
Epoch 9/10
686/686 [==============================] - 98s 143ms/step - loss: 2.2780
Epoch 10/10
686/686 [==============================] - 98s 143ms/step - loss: 2.1873


**결과)** 10 Epoch 안에 loss가 2.1873으로 2.2 수준으로 낮추어졌다.

**모델 평가하기**

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

## 모델이 생성한 문장 :

> 'i love you , i love you'

### TEST:

In [19]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m bad , i m bad you know it <end> '

In [46]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m bad , i m bad you know it <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> bad", max_len=20)

'<start> bad boy , fatty girl <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> winter", max_len=20)

'<start> winter s gone , i m the only one <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> dream", max_len=20)

'<start> dream , i m ready <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> cool", max_len=10)

'<start> cool , easy , easy <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> snow", max_len=20)

'<start> snow in the sky <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> movie", max_len=20)

'<start> movie <end> '

In [42]:
generate_text(model, tokenizer, init_sentence="<start> how", max_len=20)

'<start> how many times i gotta do ? <end> '

In [44]:
generate_text(model, tokenizer, init_sentence="<start> how are", max_len=20)

'<start> how are you ? <end> '

In [45]:
generate_text(model, tokenizer, init_sentence="<start> what", max_len=20)

'<start> what you want nixga what you what you want nixga <end> '

## 결과

> 최종적으로 Embedding Size = 256 Hidden Size = 1000 으로 설정하고, 10 Epoch 안에 loss가 2.1873으로 2.2 수준으로 낮추어졌다. 

## 총평

> input을 다양하게 해서 테스트를 했더니, 생각보다 완결된 문장을 출력했다. 그러나 같은 단어에 대해서는 같은 문장을 출력했고, 'movie'와 같은 단어의 경우엔 추가적으로 문장이 생성되지 않았다. 'movie' 라는 단어가 dataset에 없어서 그런 것일까 생각했다. 
>
>그리고 'what' 을 넣으니,  'what you want nixga what you what you want nixga'가 출력되어 흠칫 놀랬다. (racism적 단어인 줄 알고) 더불어 얼마전 논란이 된 AI 챗봇 '이루다'가 떠올랐다. '이루다' 또한 기본적으로 이러한 매커니즘을 통해서 작동을 했을 것이다. 사용자가 던지는 질문에 대답 하기 위해서는 다양하고 규모가 큰 data가 전제되어야 한다. 더욱이 사람과 대화하는 듯한 느낌을 주기 위해서는 실제 사람들의 대화 내용을 기반으로 하는 것이 논리적인 선택이었을 것이다. 그러나 이번 논란에서도 그 이전의 논란에서도 많은 데이터가 필요한 만큼 어떤 데이터를 사용하는가도 동시에 중요한 문제가 된다. AI는 도덕적 판단을 하는 것이 아니라 데이터를 기반으로 하나의 확률을 출력한 것이기 때문이다. 데이터의 정제는 여전히 사람의 영역인 듯 하다. 
>
>good data가 good output으로 이어진다.  